In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from datetime import datetime
import optuna
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.arima.model import ARIMA

c:\Users\Admin\.conda\envs\python\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import optuna
from data_utils import *

In [183]:
train_df = pd.read_csv('../__input__/1_DataPreprocessing/train_df.csv')
test_df = pd.read_csv('../__input__/1_DataPreprocessing/valid_df.csv')
train_df['Date'] = pd.to_datetime(train_df['Date'])
test_df['Date'] = pd.to_datetime(test_df['Date'])
test_df.head()

,Date,SecuritiesCode,ExpectedDividend,AdjustmentFactor,SupervisionFlag,Volume,Open,High,Low,Close,Target,CloseT1,CloseT2
0,2021-05-28,1301,-1.0,1.0,False,96000,2905.0,2909.0,2851.0,2869.0,0.026934,2896.0,2974.0
1,2021-05-28,1332,-1.0,1.0,False,1578600,509.0,517.0,508.0,514.0,0.013807,507.0,514.0
2,2021-05-28,1333,-1.0,1.0,False,148900,2405.0,2420.0,2400.0,2415.0,0.016701,2395.0,2435.0
3,2021-05-28,1375,-1.0,1.0,False,156900,1781.0,1797.0,1764.0,1770.0,-0.006384,1723.0,1712.0
4,2021-05-28,1376,-1.0,1.0,False,8300,1540.0,1540.0,1506.0,1524.0,0.006098,1476.0,1485.0


In [184]:
train_linear = train_df[train_df['Date'] > datetime(2021,2,28)].reset_index(drop = True)
test_linear = test_df[test_df['Date'] < datetime(2021,6,28)].reset_index(drop = True)

In [185]:
test_linear.tail()

,Date,SecuritiesCode,ExpectedDividend,AdjustmentFactor,SupervisionFlag,Volume,Open,High,Low,Close,Target,CloseT1,CloseT2
41773,2021-06-25,9990,-1.0,1.0,False,59900,655.0,658.0,649.0,657.0,-0.032012,656.0,635.0
41774,2021-06-25,9991,-1.0,1.0,False,21100,909.0,910.0,900.0,906.0,-0.015470,905.0,891.0
41775,2021-06-25,9993,-1.0,1.0,False,1300,1734.0,1734.0,1734.0,1734.0,-0.008060,1737.0,1723.0
41776,2021-06-25,9994,-1.0,1.0,False,6100,2292.0,2292.0,2262.0,2262.0,0.000880,2274.0,2276.0
41777,2021-06-25,9997,-1.0,1.0,False,457400,969.0,987.0,962.0,983.0,-0.012384,969.0,957.0


In [175]:
train_linear.tail()

,Date,SecuritiesCode,ExpectedDividend,AdjustmentFactor,SupervisionFlag,Volume,Open,High,Low,Close,Target,CloseT1,CloseT2
113492,2021-05-27,9990,-1.0,1.0,False,27900,576.0,584.0,575.0,575.0,0.026756,598.0,614.0
113493,2021-05-27,9991,-1.0,1.0,False,306000,863.0,894.0,860.0,874.0,-0.023438,896.0,875.0
113494,2021-05-27,9993,-1.0,1.0,False,3100,1700.0,1714.0,1690.0,1690.0,-0.005266,1709.0,1700.0
113495,2021-05-27,9994,-1.0,1.0,False,4300,2291.0,2291.0,2280.0,2280.0,-0.004806,2289.0,2278.0
113496,2021-05-27,9997,-1.0,1.0,False,340700,1113.0,1119.0,1078.0,1082.0,-0.014493,1104.0,1088.0


In [176]:
train_linear = pd.concat([train_linear,test_linear])

In [177]:
def predict(df,num_lags,date):
    codes = df['SecuritiesCode'].unique()
    pred=[]
    reg = linear_model.LinearRegression()
    for code in codes:
        his = df[df['SecuritiesCode'] == code].reset_index()
        print(code)
        X = np.arange(1,num_lags+1).reshape(num_lags,1)
        y = his[his['Date'] == date][[f'CloseT{i}' for i in range(1,num_lags+1,1)]].values[0]
        reg.fit(X,y)
        y_pred = reg.predict([[num_lags+1]])[0]
        pred.append(y_pred)
    return pred

In [178]:
date_test_start = pd.to_datetime(test_linear['Date'].values[0])
date_test_end = pd.to_datetime(test_linear['Date'].values[-1])
COLUMN_NAMES = ['SecuritiesCode','Close','Close_pred','Date']
NUMS_LAG = 1 
def make_prediction(df,nums_lag,date_start,date_end):
    new_df = pd.DataFrame(columns=COLUMN_NAMES)
    codes = df['SecuritiesCode'].unique()
    nums_day = (date_end-date_start).days
    for i in range(nums_day+1):
        date  = date_start + pd.Timedelta(f"{i} day")
        y_true = df[df['Date'] == date]['Close'].values
        if len(y_true) == 0:
            continue
        prediction = predict(df,nums_lag,date)
        dates = [date for _ in range(len(codes))]
        data = {'SecuritiesCode':codes,'Close_pred':prediction,'Close':y_true,'Date':dates}
        new_df = pd.concat([new_df,pd.DataFrame(data)])
    return new_df

In [182]:
train_linear[(train_linear['SecuritiesCode'] == 2761)]['Date'].values

array(['2021-03-17T00:00:00.000000000', '2021-03-18T00:00:00.000000000',
       '2021-04-26T00:00:00.000000000', '2021-05-13T00:00:00.000000000',
       '2021-05-14T00:00:00.000000000', '2021-05-17T00:00:00.000000000',
       '2021-05-18T00:00:00.000000000', '2021-06-08T00:00:00.000000000',
       '2021-06-09T00:00:00.000000000', '2021-06-10T00:00:00.000000000',
       '2021-06-11T00:00:00.000000000', '2021-06-23T00:00:00.000000000',
       '2021-06-24T00:00:00.000000000', '2021-06-25T00:00:00.000000000'],
      dtype='datetime64[ns]')

In [170]:
# train_linear = pd.concat([train_df,test_df])
pred_df = make_prediction(train_linear,NUMS_LAG,date_test_start,date_test_end)
pred_df.head()

1301
1332
1333
1376
1377
1379
1381
1407
1414
1417
1419
1429
1435
1515
1518
1605
1662
1663
1712
1716
1719
1720
1721
1723
1726
1762
1766
1775
1780
1787
1793
1799
1801
1802
1803
1805
1808
1810
1811
1812
1813
1814
1815
1820
1821
1822
1833
1835
1852
1860
1861
1870
1871
1873
1878
1879
1882
1884
1885
1888
1890
1893
1898
1899
1911
1914
1921
1925
1926
1928
1929
1930
1934
1938
1939
1941
1942
1944
1945
1946
1949
1950
1951
1952
1954
1959
1961
1963
1965
1967
1968
1969
1973
1975
1976
1979
1980
1981
1982
2001
2002
2003
2004
2009
2053
2060
2108
2109
2114
2117
2120
2121
2124
2127
2130
2146
2148
2150
2153
2154
2157
2158
2160
2168
2170
2175
2181
2183
2185
2193
2198
2201
2204
2206
2207
2208
2209
2211
2212
2217
2220
2221
2222
2226
2229
2264
2266
2267
2268
2269
2270
2281
2282
2288
2292
2294
2296
2301
2305
2307
2309
2315
2317
2325
2326
2327
2329
2331
2335
2337
2349
2353
2359
2371
2372
2374
2378
2379
2384
2389
2393
2395
2412
2413
2418
2427
2429
2432
2433
2440
2445
2453
2461
2462
2469
2471
2475
2477
2484
2489


IndexError: index 0 is out of bounds for axis 0 with size 0